In [4]:
"""
This is the main file that runs the simulation. The simulation is output within this file, and all code cells should
be invisible to the user. 
"""
import generate_gaussian as rg
import continuous_growth as cog
import generate_cycles as cyg
import generate_spikes as sg
import pandas as pd
import ast
import ipywidgets as ipy
import numpy as np
import aux

In [5]:
"""
Reads in data from the given csv file and converts inputs meant to be lists into lists.
Parameters:
    csv(str): The name of a csv file.
Returns:
    graphs(pandas dataframe): A dataframe where each row contains specifications for a graph to be generated. 
"""
def process_data(csv):
    graphs = pd.read_csv(csv, na_values = "None")
    graphs = graphs.replace(np.nan, None)
    
    #convert to lists where appropriate
    for i in range(len(graphs["type_var3"])):
        lists = graphs["type_var3"][i]
        if lists:
            #lists are input in the csv with spaces instead of commas so they aren't read as multiple columns
            lists = lists.replace(" ", ",")
            #use ast to translate this string to a Python list
            custom_cycles = ast.literal_eval(lists)
            #set the list as the new val at that space
            graphs.at[i, "type_var3"] = custom_cycles
            
    #repeat above process for custom distributions, input in the same way
    for i in range(len(graphs["custom_dist"])):
        dist = graphs["custom_dist"][i]
        if dist:
            dist = dist.replace(" ", ",")
            custom_dist = ast.literal_eval(dist)
            graphs.at[i, "custom_dist"] = custom_dist
            
    return graphs

graphs = process_data("graph_data.csv")
print(graphs)

         type  mean  sd  time type_var1 type_var2  \
0      random   100   5   168      None      None   
1      cycles   100   5   168     200.0       5.0   
2  continuous     0   2   365    1000.0       2.0   
3      cycles   200   5   168     300.0       5.0   
4      random   200  10   210      None      None   
5  continuous   200  20   365     200.0       1.5   
6      spikes   100   5   168     200.0      0.02   
7      spikes   100   5   210     200.0      0.05   
8      spikes   100   5   365     200.0       0.2   

                        type_var3             custom_dist  graph  id timeframe  
0                            None  [[0.5, 0.9], [1, 0.1]]  False   1     daily  
1                 [[100, 168, 0]]                    None  False   1   monthly  
2                               1                    None  False   1    yearly  
3  [[35, 42, 56], [154, 167, 21]]                    None  False   2     daily  
4                            None                    None  False

In [6]:
#a dictionary that stores run keys in the form "id-timeframe" to distributions, so that while each
#session generates a new random distribution, switching between timeframes / problems does not
hasRun = dict()

"""
Displays the graph with the input id and timeframe, or if that id and timeframe combination has been run before, 
uses the generated data for that combination.
Note: every id and timeframe combination should be unique, e.g. there should not be two 'daily' graphs at problem 2.
Parameters:
    id(int): The number that corresponds to the problem/question that the graphs are presented in.
    timeframe(str): The time over which the graph takes place. Valid inputs are 'daily', 'monthly', and 'yearly'.
"""
def run_simulation(id, timeframe):
    #determine which row numbers contain a graph with the searched for id
    sim_graphs = []
    for row in range(len(graphs["id"])):
        if id == graphs["id"][row]:
            sim_graphs.append(row)
    
    #for each graph with that id
    for row_id in sim_graphs:
        #check if it is the proper timeframe
        if timeframe == graphs['timeframe'][row_id]:
            #create the run key to be checked in hasRun
            runKey = (str(id) + "-" + timeframe)
            #if hasRun.keys() contains the run key, then this id timeframe combo has been run before in this session.
            if runKey in hasRun.keys():
                #do not generate a new distribution. Simply plot the distribution stored. 
                requests_y = hasRun[runKey]
                time_x = [i for i in range(graphs["time"][row_id])]
                aux.lim_plots(time_x, requests_y)
            #if this combination has not been run
            else:
                #determine type and run the corresponding function
                graph_type = graphs["type"][row_id]
                if graph_type == "random":
                    requests_y = rg.generate_random(graphs["mean"][row_id], graphs["sd"][row_id], graphs["time"][row_id], 
                                      graphs["custom_dist"][row_id], graphs["graph"][row_id])
                elif graph_type == "continuous":
                    requests_y = cog.generate_continuous(graphs["mean"][row_id], graphs["sd"][row_id], graphs["time"][row_id], 
                                      int(graphs["type_var1"][row_id]), int(graphs["type_var2"][row_id]),
                                            graphs["type_var3"][row_id], graphs["custom_dist"][row_id], 
                                            graphs["graph"][row_id])
                elif graph_type == "cycles":
                    requests_y = cyg.cycles_gen(graphs["mean"][row_id], graphs["sd"][row_id], graphs["time"][row_id], 
                                      graphs["type_var1"][row_id], graphs["type_var2"][row_id], 
                                        graphs["type_var3"][row_id], graphs["custom_dist"][row_id], 
                                       graphs["graph"][row_id])
                elif graph_type == "spikes":
                    requests_y = sg.generate_spikes(graphs["mean"][row_id], graphs["sd"][row_id], graphs["time"][row_id], 
                                      int(graphs["type_var1"][row_id]), graphs["type_var2"][row_id],
                                            graphs["custom_dist"][row_id], graphs["graph"][row_id])
                else:
                    print("Error in CSV file: type at row " + str(row_id) + " is not a valid type.")
                    break
                #then store that new list in hasRun with the run key.
                time_x = [i for i in range(graphs["time"][row_id])]
                aux.lim_plots(time_x, requests_y)
                hasRun[runKey] = requests_y

#create interactable version of above function. 
ipy.interact(run_simulation, timeframe = ['daily', 'monthly','yearly'], id = [1,2,3])


interactive(children=(Dropdown(description='id', options=(1, 2, 3), value=1), Dropdown(description='timeframe'…

<function __main__.run_simulation(id, timeframe)>